# Stamps Problem

This notebook covers my solution to the following problem:

A trader has discovered a source of 1000 unique types of stamps which she can buy in bulk and then sell to a network of 100 merchants through an intermediary. No matter what type of stamp, she pays $1 for each stamp. Each day, she can sell up to 100 bags of stamps (each bag containing one or more different stamps, with no limit on how many stamps can be in each bag), and she is paid the next day. Unfortunately, the intermediary hides the prices of the individual stamps, and only tells the trader the per-bag price. In order to consistently turn a profit at a reasonable margin, it would be helpful to know which of the 1000 types of stamps are the most valuable, so she could sell bags with only the most valuable stamps. Given 7 days to evaluate 1000 types of stamps using 100 bags/day max, how does she figure out which types of stamps are the most valuable?

### Imports

In [33]:
from scipy.stats import truncnorm
from random import shuffle
from collections import namedtuple
from itertools import chain
from typing import List
import math

### Constants

- __low_bound__: The lower limit of a stamp's value.
- __upper_bound__: The upper limit of a stamp's value.
- __mu__: Average value of a stamp.
- __sd__: Standard deviation of the distribution of stamp values.
- __N_STAMPS__: How many stamps to use in a simulation.
- __N_SIMULATION__: How many simulations to perform.

In [34]:
# Stamp value distribution paramaters
low_bound = 0
upper_bound = 100
mu = 1
sd = 0.6

# Problem variables
N_STAMPS = 1_000
N_SIMULATIONS = 1_000
N_BAGS_PER_DAY = 100

# Solution Variables
PHASE_1_DURATION = 4
PHASE_2_DURATION = 1

### Modeling the Problem

Creates some data structures and a class to model the problem, making it easier to test different solutions.

In [35]:
Stamp = namedtuple("Stamp", ['id', 'value'])
BagSample = namedtuple("BagSample", ["stamp_ids", "bag_value"])

class StampEnv:
    def __init__(self, n_stamps):
        a, b = (low_bound - mu) / sd, (upper_bound - mu) / sd
        stamp_values = truncnorm.rvs(a, b, loc=mu, scale=sd, size=n_stamps)
        stamp_values = (stamp_values * 20).round() / 20

        self.stamps = [Stamp(id=i, value=v) for i, v in enumerate(stamp_values)]
        self.day = 0

    def sample(self, bags: List[List[int]]):
        bag_values = [sum([self.stamps[stamp_id].value for stamp_id in bag]) for bag in bags]
        self.day += 1
        return bag_values

### Solution

The strategy I used to solve this problem has two parts. First, all 1,000 stamps are sampled once over 5 days which requires placing 2 stamps per bags, for a total of 500 bags. Of these, only the top 100 values bags are considered, leaving a candidate pool of 200 stamps. Over the final two days, each stamp can be sent in it's own bag allowed the exact value to be determined. Whichever stamp performs best of this group of 200 stamps is the solution to this problem.

In [46]:
def simulate():
    env = StampEnv(N_STAMPS)
    collected_samples = []
    PHASE_1_STAMPS_PER_BAG = math.ceil(N_STAMPS / (PHASE_1_DURATION * N_BAGS_PER_DAY))

    for day in range(PHASE_1_DURATION):
        day_shift = day * PHASE_1_STAMPS_PER_BAG * N_BAGS_PER_DAY

        for bag in range(N_BAGS_PER_DAY):
            stamp_ids = list(range(day_shift + (bag * PHASE_1_STAMPS_PER_BAG), day_shift + ((bag + 1) * PHASE_1_STAMPS_PER_BAG)))
            
            # This method can be wasteful depending on the parameters, leading to unused bags.
            # If I was making this production ready, I would use the remaining bags to begin
            # the phase 2 process.
            stamp_ids = [stamp_id for stamp_id in stamp_ids if stamp_id < N_STAMPS]
            
            bag_value = env.sample([stamp_ids])[0]
            collected_samples.append(BagSample(stamp_ids=tuple(stamp_ids), bag_value=bag_value))

    collected_samples.sort(key=lambda x: x.bag_value, reverse=True)
    bags_remaining = PHASE_2_DURATION * N_STAMPS_PER_BAG
    top_samples = collected_samples[:bags_remaining]
    candidates = list(chain(*[x.stamp_ids for x in top_samples]))

    final_results = env.sample([[x] for x in candidates])

    best_stamp_found_id = candidates[final_results.index(max(final_results))]
    best_stamp_found = env.stamps[best_stamp_found_id]
    best_stamp_actual = max(env.stamps, key=lambda x: x.value)

    return best_stamp_found, best_stamp_actual

### Testing Solution

Results will vary between runs, but the best stamp should be discovered about 99.8% of the time. When the best stamp is not selected, this algorithm is not very far off, usully averaging just 10 cents in lost value.

How effective this method is will change based on the stamp value distribution. Testing how this method performs on other distributions such as uniform, exponantial, or skewed normal should be evaluated as well.

In [47]:
best_found = 0
lost_value = []
for _ in range(N_SIMULATIONS):
    pred, best = simulate()
    if pred.value == best.value:
        best_found += 1
    else:
        lost_value.append(best.value - pred.value)
        
avg_lost_value = sum(lost_value) / len(lost_value) if len(lost_value) > 0 else None
        
print(f"Out of {N_SIMULATIONS:,} simulations, the best stamp was found {best_found:,} times.")
if avg_lost_value is not None:
    print(f"When the best stamp was not selected, it was on average ${avg_lost_value:,.2} lower in value than the actual best.")

Out of 1,000 simulations, the best stamp was found 973 times.
When the best stamp was not selected, it was on average $0.17 lower in value than the actual best.
